In [1]:
cd ..

m:\OneDrive\Projects\federated_imputation_pc2


## Loading

In [2]:
%load_ext autoreload
%autoreload
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from dython.nominal import correlation_ratio
import missingno as msno
from src.fed_imp.sub_modules.missing_simulate.missing_adder_new import simulate_nan_new
from src.modules.evaluation.imputation_quality import rmse, ws_cols, sliced_ws

In [9]:
experiment_config_template = {
  "num_clients": 10,
  "data": {
    "dataset_name": "codrna",
    "normalize": True,
  },
  "handle_imbalance": None,
  "data_partition": {
    "strategy": 'sample-evenly',
    "params": {}
  },
  "missing_simulate": {
    "mr_strategy": "random",
    "mf_strategy": "all",
    "mm_strategy": "single@m=1",
    "mm_strategy_new": ""
  },
  "imputation": {
    "initial_strategy": "mean",
    "clip": True,
    "imp_model_params":{
      "loss_alpha": 10,
    },
    "imp_params":{
      "lr": 1e-3,
      "batch_size": 256,
      "weight_decay": 0.0001, 
      "hint_rate": 0.3,
      "loss_alpha": 10, 
      "optimizer": "adam"
    }
  },
  "agg_strategy_imp": {
    "strategy": 'local_gain',
    "params": {
      "ms_field": "missing_cell_pct",
      "beta": 0.7
    }
  },
  "client_type": "gain",
  "server_type": "gain",
  "server": {
    "impute_round": 600,
    "imp_local_epochs": 5,
    "pred_round": 0,
    "pred_local_epochs": 0,
    "verbose": 5
  },
  "pred_model": {
    "model_params": {
      "model": "2nn",
      "num_hiddens": 32,
      "model_init_config": None,
      "model_other_params": None
    },
    "train_params": {
      "batch_size": 128,
      "learning_rate": 0.001,
      "weight_decay": 0.0000,
      "pred_round": 200,
      "pred_local_epochs": 3
    }
  },
  "experiment": {
    "n_rounds": 1,
    "seed": 102931466,
    "mtp": False,
    "random_seed": 50,
    "num_process": 3,
    "test_size": 0.1,
    "track": True
  },
  "tune_params": False,
  "track": False,
  "prediction": False,
  "save_state": False,
  "test_size": 0.1,
  "algo_params": {
    "central": {},
    "central2": {},
    "central_vae": {},
    "central_gain": {},
    "local": {},
    "local_vae": {},
    "local_gain": {},
    "fedavg-s": {},
    "fedavg_vae": {},
    "fedavg_gain": {},
    "fedavg": {},
    "fedwavg": {
      "alpha": 0.9
    },
    "fedmechw": {
      "client_thres": 1.0,
      "alpha": 1.0,
      "beta": 0.0
    },
    "fedmechclw": {
      "client_thres": 0.2,
      "thres1": 0.2,
      "alpha": 0.5
    },
    'testavg':{
        
    },
    'testavg2':{
        
    },
    'testavg3':{
        
    },
    "scale_factor": 4
  }
}

server_config_tmpl = {
    "server_name": 'fedavg_mlp_pytorch_pred',
    "server_pred_config": {
        "model_params": {
            "model": "2nn",
            "num_hiddens": 128,
            "model_init_config": None,
            "model_other_params": None
        },
        "train_params": {
            "batch_size": 300,
            "learning_rate": 0.001,
            "weight_decay": 0.0,
            "pred_round": 1000,
            "pred_local_epochs": 5,
            'local_epoch': 5,
            'sample_pct': 1
        }
    },
    "server_config": {
        'pred_rounds': 1,
        'seed': 21
    }
}

In [10]:
trackers = {}
rets ={}
mr_strategy = 'fixed@mr=0.5'

In [11]:
import dotenv
import os

# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv(".env", override=True)

print('MY_VAR = ', os.environ.get('CUBLAS_WORKSPACE_CONFIG')) # MY_VAR = HELLO_BOB

MY_VAR =  :4096:8


In [14]:
%load_ext autoreload
%autoreload
%load_ext dotenv
%dotenv
from src.fed_imp.experiment import Experiment

exp = Experiment()
exp.run_experiment(experiment_config_template)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
{'num_clients': 10, 'data': {'dataset_name': 'codrna', 'normalize': True}, 'handle_imbalance': None, 'data_partition': {'strategy': 'sample-evenly', 'params': {}}, 'missing_simulate': {'mr_strategy': 'random', 'mf_strategy': 'all', 'mm_strategy': 'single@m=1', 'mm_strategy_new': ''}, 'imputation': {'initial_strategy': 'mean', 'clip': True, 'imp_model_params': {'loss_alpha': 10, 'h_dim': 8}, 'imp_params': {'lr': 0.001, 'batch_size': 256, 'weight_decay': 0.0001, 'hint_rate': 0.3, 'loss_alpha': 10, 'optimizer': 'adam', 'clip': True}}, 'agg_strategy_imp': {'strategy': 'local_gain', 'params': {'ms_field': 'missing_cell_pct', 'beta': 0.7}}, 'client_type': 'gain', 'server_type': 'gain', 'server': {'impute_round': 600, 'imp_local_epochs': 5, 'pred_round': 0, 'pred_local_epochs': 0, 'verbose': 5, 'n_cols': 8}, 'pred_model': {'mod

2024-05-14 16:21:44.460 | INFO     | src.fed_imp.experiment:main_func:31 - Round 0


(18000, 9)
sample-evenly {}
['mcar@0.5038779294458209', 'mcar@0.5489826094231575', 'mcar@0.36323746467048146', 'mcar@0.49527322553523556', 'mcar@0.3474779809125788', 'mcar@0.6390912066453927', 'mcar@0.3852361537062045', 'mcar@0.5255241291045322', 'mcar@0.6278036464868088', 'mcar@0.46087350917459124']
Adding missing to client 0 ...
Adding missing to client 1 ...
Adding missing to client 2 ...
Adding missing to client 3 ...
Adding missing to client 4 ...
Adding missing to client 5 ...
Adding missing to client 6 ...
Adding missing to client 7 ...
Adding missing to client 8 ...
Adding missing to client 9 ...
Used default params: {}


2024-05-14 16:21:47.063 | INFO     | src.fed_imp.sub_modules.server.server_gain:run:124 - Server Round: 0 avg_rmse: 0.16871070020088455
  0%|          | 0/100 [00:00<?, ?it/s]2024-05-14 16:21:47.066 | INFO     | src.fed_imp.sub_modules.server.server_gain:run:133 - ==================================================
2024-05-14 16:21:47.066 | INFO     | src.fed_imp.sub_modules.server.server_gain:run:134 - Imputation Round 1
2024-05-14 16:21:49.864 | INFO     | src.fed_imp.sub_modules.server.server_gain:_run_round_impute:281 - Server Round: 1 avg_rmse: 0.20058438613754048 avg_g_loss: 0.7695252840975696 avg_d_loss: 0.7157310402393342
 35%|███▌      | 35/100 [00:36<01:07,  1.04s/it]


KeyboardInterrupt: 

In [ ]:
# def run_impute(strategy, params, dataset = 'mimiciii_mo2'):
#     name = f'mnar_lr_{dataset}_{strategy}'
#     config2 = experiment_config_template.copy()
#     config2['data']['dataset_name'] = 'mimiciii_mo2'
#     config2['handle_imbalance'] = None
#     config2['num_clients'] = 11
#     config2['missing_simulate']['mr_strategy'] = mr_strategy
#     config2['missing_simulate']['mf_strategy'] = 'all'
#     config2['missing_simulate']['mm_strategy'] = 'mnar_lr@sp=extremer1'
#     config2['algo_params'][strategy] = params
#     config2['algo_params']['scale_factor'] = params['scale_factor']
#     config2['data_partition']['strategy'] = 'sample-evenly'

#     config2['agg_strategy_imp']['strategy'] = strategy
#     config2['server_type'] = 'fedavg_pytorch'
#     config2['prediction'] = False
#     config2['save_state'] = True
#     config2['track'] = True

#     clients, test_data, new_seed = simulate_scenario(config2)
#     server, ret = run_simulation(config2, clients, test_data, new_seed)
#     trackers[name] = copy.deepcopy(server.stats_tracker)
#     rets[name] = ret.copy()

# params = { "client_thres": 1.0, "alpha": 1.0, "beta": 0.0}

# MIMIC III MORTALITY

In [32]:
def run_impute(strategy, params, dataset = 'mimiciii_mo2'):
    name = f'mnar_lr_{dataset}_{strategy}'
    config2 = experiment_config_template.copy()
    config2['data']['dataset_name'] = 'mimiciii_mo2'
    config2['handle_imbalance'] = None
    config2['num_clients'] = 11
    config2['missing_simulate']['mr_strategy'] = mr_strategy
    config2['missing_simulate']['mf_strategy'] = 'all'
    config2['missing_simulate']['mm_strategy'] = 'mnar_lr@sp=extremer1'
    config2['algo_params'][strategy] = params
    config2['algo_params']['scale_factor'] = params['scale_factor']
    config2['data_partition']['strategy'] = 'sample-evenly'

    config2['agg_strategy_imp']['strategy'] = strategy
    config2['server_type'] = 'fedavg_pytorch'
    config2['prediction'] = False
    config2['save_state'] = True
    config2['track'] = True

    clients, test_data, new_seed = simulate_scenario(config2)
    server, ret = run_simulation(config2, clients, test_data, new_seed)
    trackers[name] = copy.deepcopy(server.stats_tracker)
    rets[name] = ret.copy()

params = { "client_thres": 1.0, "alpha": 1.0, "beta": 0.0}

In [39]:
params = { "client_thres": 1.0, "alpha": 1.0, "beta": 0.0, 'scale_factor': 3}

for alpha in [0.9]:
    print("=============================================================")
    params['alpha'] = alpha
    params['beta'] = 1.0 - alpha
    run_impute('fedmechw_sigmoid', params, 'mimiciii_mo2')

(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 10:24:02.530 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:24:02.531 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 10:26:08.569 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:26:08.569 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 10:28:24.882 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:28:24.882 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 10:28:38.766 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 278.94807869999204


{'imp@rmse': 0.051653288277240396, 'imp@ws': 0.4132374405712598, 'imp@sliced_ws': 0.008140255084459268}


In [38]:
params = { "client_thres": 1.0, "alpha": 1.0, "beta": 0.0, 'scale_factor': 3}

for alpha in [1.0, 0.8, 0.6, 0.4, 0.2, 0.1]:
    print("=============================================================")
    params['alpha'] = alpha
    params['beta'] = 1.0 - alpha
    run_impute('fedmechw_sigmoid', params, 'mimiciii_mo2')

(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:42:31.559 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:42:31.560 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:44:25.197 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:44:25.198 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:46:30.633 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:46:30.633 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:46:43.599 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 254.75982399994973


{'imp@rmse': 0.051656190597141036, 'imp@ws': 0.41320757726942886, 'imp@sliced_ws': 0.008141858896719506}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:46:46.790 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:46:46.790 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:48:36.735 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:48:36.735 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:50:44.460 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:50:44.461 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:50:58.431 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 254.31083300011232


{'imp@rmse': 0.051657306608311304, 'imp@ws': 0.41316501397070093, 'imp@sliced_ws': 0.008147177352508408}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


...

{'imp@rmse': 0.051911910578992554, 'imp@ws': 0.41607144667485907, 'imp@sliced_ws': 0.00914469835341828}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:59:28.862 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:59:28.863 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 10:01:14.411 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:01:14.412 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 10:03:21.558 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:03:21.559 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 10:03:34.587 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 248.31141109997407


{'imp@rmse': 0.0615629023336361, 'imp@ws': 0.6164410764056489, 'imp@sliced_ws': 0.022333695281212068}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 10:03:37.793 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:03:37.794 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 10:05:23.508 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:05:23.509 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 10:07:22.439 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 10:07:22.440 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 10:07:34.574 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 239.46863810007926


{'imp@rmse': 0.1088515037730871, 'imp@ws': 1.5641324318338128, 'imp@sliced_ws': 0.05601844987085222}


In [36]:
params = { "client_thres": 1.0, "alpha": 0.9, "beta": 0.1, 'scale_factor': 3}

for client_thres in [0.2,0.4,0.6,0.8,1.0]:
    print("=============================================================")
    params['client_thres'] = client_thres
        
    run_impute('fedmechw', params, 'mimiciii_mo2')

(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:20:40.727 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:20:40.728 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:22:37.917 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:22:37.917 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:24:41.757 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:24:41.757 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:24:54.344 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 256.35838240000885


{'imp@rmse': 0.051756839434598176, 'imp@ws': 0.41395357168107905, 'imp@sliced_ws': 0.008270551951529767}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:24:57.423 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:24:57.424 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:26:45.396 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:26:45.397 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:28:46.746 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:28:46.747 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:28:59.218 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 244.411931599956


{'imp@rmse': 0.05161272311468685, 'imp@ws': 0.41414701791805486, 'imp@sliced_ws': 0.008194286301782512}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


...

{'imp@rmse': 0.05174226900979438, 'imp@ws': 0.4128424394791663, 'imp@sliced_ws': 0.008203271821592552}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:33:06.532 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:33:06.533 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:34:53.780 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:34:53.781 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:37:01.175 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:37:01.176 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:37:13.703 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 249.8405899000354


{'imp@rmse': 0.051636093824684905, 'imp@ws': 0.41333302591909077, 'imp@sliced_ws': 0.008128932137486366}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:37:16.815 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:37:16.816 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:39:08.298 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:39:08.299 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:41:15.866 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:41:15.866 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:41:29.072 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 254.92458460002672


{'imp@rmse': 0.05165635451728335, 'imp@ws': 0.41319585527379504, 'imp@sliced_ws': 0.008142314277959533}


In [35]:
params = { "client_thres": 1.0, "alpha": 0.9, "beta": 0.1, 'scale_factor': 1}

for scale_factor in [1,2,4,6,8]:
    print("=============================================================")
    params['scale_factor'] = scale_factor
    run_impute('fedmechw', params, 'mimiciii_mo2')

(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 08:54:07.565 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 08:54:07.566 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 08:56:12.684 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 08:56:12.685 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 08:58:28.079 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 08:58:28.080 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 08:58:41.730 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 276.8815225000726


{'imp@rmse': 0.06021078491422165, 'imp@ws': 0.5890107334061735, 'imp@sliced_ws': 0.019744459539088266}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 08:58:44.815 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 08:58:44.815 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:00:34.020 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:00:34.021 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:02:42.150 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:02:42.151 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:02:55.569 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 253.38098040001933


{'imp@rmse': 0.05167819946915961, 'imp@ws': 0.4119608649272319, 'imp@sliced_ws': 0.008191949479345735}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


...

{'imp@rmse': 0.05165832137345263, 'imp@ws': 0.41320749290067915, 'imp@sliced_ws': 0.008143773224991798}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:07:11.483 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:07:11.484 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:08:58.911 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:08:58.912 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:11:09.679 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:11:09.681 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:11:22.658 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 253.80522929993458


{'imp@rmse': 0.05166375816515767, 'imp@ws': 0.41319301571493877, 'imp@sliced_ws': 0.008149856205004828}
(20000, 51)
mortality
0    14556
1     5444
Name: count, dtype: int64
(18000, 51) (2000, 51)
(18000, 51)
sample-evenly {}
['mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_left@0.5', 'mnar_quantile_right@0.5']


2023-08-23 09:11:25.801 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:11:25.802 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 1
2023-08-23 09:13:17.554 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:13:17.555 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 10
2023-08-23 09:15:27.428 | INFO     | src.fed_imp.sub_modules.server.base_server:run:148 - ==================================================
2023-08-23 09:15:27.429 | INFO     | src.fed_imp.sub_modules.server.base_server:run:149 - Imputation Round 20
2023-08-23 09:15:40.271 | INFO     | src.fed_imp.sub_modules.server.base_server:run:178 - FL finished in 257.15209810005035


{'imp@rmse': 0.05167047287157184, 'imp@ws': 0.41318864728681126, 'imp@sliced_ws': 0.008159208246406414}
